In [3]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

fn = 'raw_data/loco_20170210_03.mat'
# 加载.mat文件
with h5py.File(fn, 'r') as file:
    finger_pos = file['finger_pos'][:]
    finger_pos = np.array(finger_pos).transpose()
    cursor_pos = file['cursor_pos'][:]
    cursor_pos = np.array(cursor_pos).transpose()
    t = file['t'][:]
    t = np.array(t).transpose()
    print(finger_pos.shape)
    print(t.shape)
    spikes = []
    for column in file['spikes']:
        column_data = []
        for row_number in range(len(column)):
            row_data = np.array(file[column[row_number]])
            column_data.append(row_data)
        spikes.append(column_data)
    spikes = np.array(spikes, dtype='object').transpose()
    print(spikes.shape)


(442095, 3)
(442095, 1)
(192, 5)


In [4]:
spikes[0][1].shape[1]

1715

In [86]:
import numpy as np
import matplotlib.pyplot as plt
import os

fn = 'tuning_curve_speed'
if not os.path.exists(fn):
    os.makedirs(fn)

for neuron_row in range(96):
    for neuron_col in range(1, 5):
        # 导入spikes
        spike_times = spikes[neuron_row][neuron_col]
        if spike_times.shape[0] == 2 or spike_times.shape[1] < 1000:
            break
        # 计算速度
        finger_pos_diff = np.diff(finger_pos, axis=0)
        finger_speed = np.sqrt(np.sum(finger_pos_diff ** 2, axis=1))

        # 定义速度的区间（bins）
        speed_bins = np.linspace(finger_speed.min(), finger_speed.max(), num=100)
        speed_bin_indices = np.digitize(finger_speed, speed_bins)

        # 计算每个区间的平均尖峰率
        mean_spike_rates = []
        for i in range(len(speed_bins)):
            # 找到该速度区间对应的时间点
            time_points = np.where(speed_bin_indices == i)[0]
            # 计算这些时间点的平均尖峰率
            mean_spike_rate = np.nanmean(
                [np.sum((t[i] <= spike_times) & (spike_times < t[i + 1])) for i in time_points])
            mean_spike_rate = mean_spike_rate * 250
            mean_spike_rates.append(mean_spike_rate)

        mean_spike_rates = np.nan_to_num(mean_spike_rates)

        # 找到最大发放率的点
        max_rate_index = np.argmax(mean_spike_rates)
        max_rate_speed = speed_bins[max_rate_index]
        max_rate = mean_spike_rates[max_rate_index]

        # 绘制tuning curve
        plt.figure(figsize=(10, 6))
        plt.plot(speed_bins, mean_spike_rates)
        plt.title('Tuning curve')
        plt.xlabel('Speed')
        plt.ylabel('Mean spike rate(Hz)')

        text = f'Max firing rate at speed: {max_rate_speed:.3f}'
        # 在图中标注最大发放率的点
        plt.plot(max_rate_speed, max_rate, 'ro')  # 使用红色圆点标注最大发放率的点
        plt.annotate(
            text,
            xy=(max_rate_speed, max_rate),
            xytext=(max_rate_speed, max_rate + 0.01),  # 这里的0.1是注释的位置，你可以根据需要进行调整
            # arrowprops=dict(facecolor='black', shrink=0.05),
        )
        plt.ylim([0, 1.1 * max_rate])

        # 保存图像
        plt.savefig(f'tuning_curve_speed/neuron_({neuron_row + 1},{neuron_col}).png')

        # 关闭图像，以防止所有的图像都显示出来
        plt.close()

/var/folders/fs/b49sl8gs2gx58q7k4gz6h_kc0000gn/T/ipykernel_38540/1638604581.py:29: RuntimeWarning: Mean of empty slice
  mean_spike_rate = np.nanmean([np.sum((t[i] <= spike_times) & (spike_times < t[i+1])) for i in time_points])
/var/folders/fs/b49sl8gs2gx58q7k4gz6h_kc0000gn/T/ipykernel_38540/1638604581.py:29: RuntimeWarning: Mean of empty slice
  mean_spike_rate = np.nanmean([np.sum((t[i] <= spike_times) & (spike_times < t[i+1])) for i in time_points])
/var/folders/fs/b49sl8gs2gx58q7k4gz6h_kc0000gn/T/ipykernel_38540/1638604581.py:29: RuntimeWarning: Mean of empty slice
  mean_spike_rate = np.nanmean([np.sum((t[i] <= spike_times) & (spike_times < t[i+1])) for i in time_points])
/var/folders/fs/b49sl8gs2gx58q7k4gz6h_kc0000gn/T/ipykernel_38540/1638604581.py:29: RuntimeWarning: Mean of empty slice
  mean_spike_rate = np.nanmean([np.sum((t[i] <= spike_times) & (spike_times < t[i+1])) for i in time_points])
/var/folders/fs/b49sl8gs2gx58q7k4gz6h_kc0000gn/T/ipykernel_38540/1638604581.py:29: R

In [8]:
import os

fn = 'tuning_curve_direction'
if not os.path.exists(fn):
    os.makedirs(fn)

for neuron_row in range(96):
    for neuron_col in range(1, 5):

        spike_times = spikes[neuron_row][neuron_col]
        if spike_times.shape[0] == 2 or spike_times.shape[1] < 1000:
            break

        # 计算方向
        finger_pos_diff = np.diff(cursor_pos, axis=0)
        finger_direction = np.arctan2(finger_pos_diff[:, 1], finger_pos_diff[:, 0])

        # 定义方向的区间（bins）
        direction_bins = np.linspace(-np.pi, np.pi, num=180)
        direction_bin_indices = np.digitize(finger_direction, direction_bins)

        # 计算每个区间的平均尖峰率
        mean_spike_rates = []
        for i in range(len(direction_bins)):
            # 找到该方向区间对应的时间点
            time_points = np.where(direction_bin_indices == i)[0]
            # 计算这些时间点的平均尖峰率
            mean_spike_rate = np.mean([np.sum((t[i] <= spike_times) & (spike_times < t[i + 1])) for i in time_points])
            mean_spike_rates.append(mean_spike_rate)

        # 绘制tuning curve
        plt.figure(figsize=(10, 6))
        plt.plot(direction_bins, mean_spike_rates)
        plt.title('Tuning curve')
        plt.xlabel('Direction')
        plt.ylabel('Mean spike rate')
        # 保存图像
        plt.savefig(f'tuning_curve_direction/neuron_({neuron_row + 1},{neuron_col}).png')
        # 关闭图像，以防止所有的图像都显示出来
        plt.close()



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dty